# Setup Code
**DO NOT CHANGE THIS!!**
**RUN THE CELL BELOW FIRST!!**

In [1]:
import carla
import os

client_name = os.environ.get("CLIENT_NAME", "DOES NOT EXIST")
if client_name == "DOES NOT EXIST":
    raise Exception("The environment variable for the container name of the carla server has not been set")

# Connect to the client and retrieve the world object
client = carla.Client(client_name, 2000)
world = client.get_world()

# Start Adding Code or Markdown Cells Below
Some helpful code cells are provided below that you may choose to use

In [2]:
#### SPAWN VEHICLES ####
"""
# Get the blueprint library and filter for the vehicle blueprints
vehicle_blueprints = world.get_blueprint_library().filter('*vehicle*')
# Get the map's spawn points
spawn_points = world.get_map().get_spawn_points()

# Spawn 50 vehicles randomly distributed throughout the map 
# for each spawn point, we choose a random vehicle from the blueprint library
for i in range(0,50):
    world.try_spawn_actor(random.choice(vehicle_blueprints), random.choice(spawn_points)))
"""
#### SET ALL VEHICLES TO AUTOPILOT
# for vehicle in world.get_actors().filter('*vehicle*'):
#     print("found car")
#     vehicle.set_autopilot(True)

for light in world.get_actors().filter('*traffic_light*'):
    
    print("light", light)

    boundingBox = light.bounding_box
    min_x = boundingBox.location.x - boundingBox.extent.x
    min_y = boundingBox.location.y - boundingBox.extent.y
    min_z = boundingBox.location.z - boundingBox.extent.z
    
    max_x = boundingBox.location.x + boundingBox.extent.x
    max_y = boundingBox.location.y + boundingBox.extent.y
    max_z = boundingBox.location.z + boundingBox.extent.z
    
    state = light.get_state()
    print("min: ", (min_x, min_y, min_z), ", max: ", (max_x, max_y, max_z), ", state: ", state)

light Actor(id=23, type=traffic.traffic_light)
min:  (-9.948899030685425, 37.71756821870804, 0.056954264640808105) , max:  (-4.113375902175903, 39.180945575237274, 1.943045735359192) , state:  Red
light Actor(id=22, type=traffic.traffic_light)
min:  (-11.15650749206543, 39.80343687534332, 0.05695605278015137) , max:  (-4.591544151306152, 40.75432527065277, 1.9430475234985352) , state:  Green
light Actor(id=21, type=traffic.traffic_light)
min:  (-11.693170070648193, 37.26614058017731, -0.3242630958557129) , max:  (-5.128206729888916, 38.217028975486755, 1.561828374862671) , state:  Red
light Actor(id=20, type=traffic.traffic_light)
min:  (-10.75691270828247, 26.436808586120605, 0.05695486068725586) , max:  (-4.191949367523193, 27.618741035461426, 1.9430463314056396) , state:  Green
light Actor(id=19, type=traffic.traffic_light)
min:  (-11.058073043823242, 6.813437461853027, 0.05694693326950073) , max:  (-4.493109703063965, 11.567879676818848, 1.9430384039878845) , state:  Green
light Ac